In [1]:
import xgboost as xgb
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

import seaborn as sns
sns.set(font='Yu Gothic')
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [7]:
def load_npy(hold_position):
    xy_mean = np.load("train_" + hold_position + "_glo_accel_xy_mean.npy")
    xy_var = np.load("train_" + hold_position + "_glo_accel_xy_var.npy")
    z_mean = np.load("train_" + hold_position + "_glo_accel_z_mean.npy")
    z_var = np.load("train_" + hold_position + "_glo_accel_z_var.npy")
    z_skew_kurtosis = np.load("train_" + hold_position + "_glo_accel_z_skew_kurtosis.npy")
    z_FFT = np.load("train_" + hold_position + "_glo_accel_z_amplitude_frequency.npy")
    result = np.concatenate([xy_mean.reshape([-1, 1]), xy_var.reshape([-1, 1]), z_mean.reshape([-1, 1]), z_var.reshape([-1, 1]), z_skew_kurtosis, z_FFT], axis=1)
    del xy_mean, xy_var, z_mean, z_var, z_skew_kurtosis, z_FFT
    return result

In [8]:
train_Bag = load_npy("Bag")
train_Hips = load_npy("Hips")
train_Torso = load_npy("Torso")
train_Hand = load_npy("Hand")

train_Bag.shape

(195491, 8)

In [15]:
X_train = np.concatenate([train_Bag, train_Hips, train_Torso, train_Hand], axis=0)
X_train.shape

(781964, 8)

In [14]:
Y_train = np.load("../Data/センサ別npyファイル/train/train_Bag/train_Bag_Label.npy")[:, 0]
Y_train = np.concatenate([Y_train, Y_train, Y_train, Y_train], axis=0)
Y_train.shape

(781964, 1)

In [12]:
def load_npy(hold_position):
    xy_mean = np.load("validation_" + hold_position + "_glo_accel_xy_mean.npy")
    xy_var = np.load("validation_" + hold_position + "_glo_accel_xy_var.npy")
    z_mean = np.load("validation_" + hold_position + "_glo_accel_z_mean.npy")
    z_var = np.load("validation_" + hold_position + "_glo_accel_z_var.npy")
    z_skew_kurtosis = np.load("validation_" + hold_position + "_glo_accel_z_skew_kurtosis.npy")
    z_FFT = np.load("validation_" + hold_position + "_glo_accel_z_amplitude_frequency.npy")
    result = np.concatenate([xy_mean.reshape([-1, 1]), xy_var.reshape([-1, 1]), z_mean.reshape([-1, 1]), z_var.reshape([-1, 1]), z_skew_kurtosis, z_FFT], axis=1)
    del xy_mean, xy_var, z_mean, z_var, z_skew_kurtosis, z_FFT
    return result

In [13]:
validation_Bag = load_npy("Bag")
validation_Hips = load_npy("Hips")
validation_Torso = load_npy("Torso")
validation_Hand = load_npy("Hand")

validation_Bag.shape

(28685, 8)

In [16]:
X_test = np.concatenate([validation_Bag, validation_Hips, validation_Torso, validation_Hand], axis=0)
X_test.shape

(114740, 8)

In [17]:
import pickle
Y_test = pickle.load(open("validation_Bag.binaryfile", 'rb'))[:, 0, 0]
Y_test.shape

(28685,)

In [18]:
Y_test = Y_test.reshape([-1, 1])
Y_test = np.concatenate([Y_test, Y_test, Y_test, Y_test], axis=0)
Y_test.shape

(114740, 1)

In [ ]:
model = xgb.XGBClassifier(learning_rate=0.05)
param = {
    "max_depth":[], "min_child_weight":[7,8,9]
    # "gamma":[0.01, 0.05, 0.075, 0.1]
    # "subsample":[0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 1.00], "colsample_bytree":[0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 1.00]
    # "n_estimators":[250, 500, 750], "learning_rate":[0.05, 0.1]
    # "reg_alpha":[0.1, 0.25, 0.5, 0.75]
}
grid = GridSearchCV(model, param_grid=param, cv=5, n_jobs=-1, scoring='f1_macro')
grid.fit(X_train[random_index], Y_train[random_index])

test_F1_Macro = grid.score(X_test, Y_test)
print("Best parameters: %s" % grid.best_params_)
print("Best Train F1_macro: %s" % grid.best_score_)
print("Test F1_macro: %s" % test_F1_Macro)